# Geometry operations {#geometric-operations}

## Prerequisites

In [ ]:
import pandas as pd
pd.set_option("display.max_rows", 4)
pd.set_option("display.max_columns", 6)

Packages...

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd

Sample data...

In [ ]:
seine = gpd.read_file("data/seine.gpkg")

## Introduction

## Geometric operations on vector data {#geo-vec}

### Simplification

Simplify...

In [ ]:
seine_simp = seine.simplify(2000)  # 2000 m

Plot:

In [ ]:
fig, axes = plt.subplots(ncols=2)
seine.plot(ax=axes[0])
seine_simp.plot(ax=axes[1])
axes[0].set_title("Original")
axes[1].set_title("Simplified (d=2000 m)")

Compare number of nodes:

In [ ]:
len(list(seine["geometry"].iloc[0].geoms[0].coords)) # Original

In [ ]:
len(list(seine_simp.iloc[0].coords)) # Simplified

### Centroids

...

### Buffers

...

### Affine transformations

...

### Clipping

...

### Subsetting and clipping

...

### Geometry unions

...

### Type transformations

...

## Geometric operations on raster data {#geo-ras}

### Geometric intersections

...

### Extent and origin

...

### Aggregation and disaggregation

...

### Resampling

...

## Exercises
